In [21]:
import os
import sys
import warnings; warnings.filterwarnings(action='ignore')

from copy import deepcopy

import numpy as np
import pandas as pd
import scipy.stats as ss

%matplotlib inline

# пропроцессинг
import category_encoders as ce

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from feature_engine.categorical_encoders import RareLabelCategoricalEncoder


# моделирование
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [6]:
# добавляем в sys.path директорию со скриптами
src_dir = os.path.join(os.getcwd(), '..', 'ocp')
sys.path.append(src_dir)

In [7]:
# загружаем необходимые скрипты
from data.loading import load_data
from data.saving import save_obj
from data.splitting import train_holdout_split, predefined_cv_strategy
from features.stats import tconfint_mean
from models.preprocessing import PandasSimpleImputer

%load_ext autoreload
%autoreload

# задаем константы
SEED = 26

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
train, test, numerical, categorical = load_data('../data/processed')

In [9]:
numerical, categorical = numerical.tolist(), categorical.tolist()

# Выбор бэйзлайн-модели

В EDA мы выяснили, что часть объектов (~10%) имеет пропущенные значения в большей части признакового пространства как в трейне, так и в тесте. Для того, чтобы корректно проводить кросс-валидацию на наших данных необходимо, чтобы распределения признаков на объектах было одинаково в обучающих и тестовом фолдах, поэтому стратификацию данных нужно делать не только по целевой переменной, но и по принадлежности к частям с разным распределением пропусков

In [10]:
# сразу заполним категориальные признаки значением 'unknown'
# как трэйне, так и в тесте
# потому как RareLabelCategoricalEncoder не обрабатывает признаки с пропусками
train[categorical] = train[categorical].fillna('unknown')
test[categorical] = test[categorical].fillna('unknown')

# сохраним обработанные данные
train.to_csv('../data/processed/train.csv')
test.to_csv('../data/processed/test.csv')

In [11]:
(X, X_train, X_holdout, 
 y, y_train, y_holdout, 
 stratify) = train_holdout_split(train, random_state=SEED)

In [12]:
# задаем стратегию кросс-валидации
cv = predefined_cv_strategy(X_train, stratify, random_state=SEED)

for train_index, test_index in cv.split():
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [    2     3     4 ... 26797 26798 26799] TEST: [    0     1    14 ... 26769 26782 26784]
TRAIN: [    0     1     4 ... 26796 26797 26799] TEST: [    2     3     8 ... 26792 26793 26798]
TRAIN: [    0     1     2 ... 26793 26797 26798] TEST: [    6     7    10 ... 26795 26796 26799]
TRAIN: [    0     1     2 ... 26797 26798 26799] TEST: [    9    11    15 ... 26772 26775 26787]
TRAIN: [    0     1     2 ... 26796 26798 26799] TEST: [    4     5    13 ... 26790 26791 26797]


In [13]:
# сформируем базовый пайплайн предобработки


# для категориальных переменных -

# tol - порок укрупнения редких категорий в признаках, 
# являясь относительной частотой, зависит от числа строк в данных,
# вычислим его значение, приняв за редкие категории, те на которые
# приходится меньше 20 строк в данных
tol = np.round(20 / (X_train.shape[0] * (4/5)), 5)

# RareLabelCategoricalEncoder - для укрупнения редких категорий по порогу,
# SimpleImputer - для обработки пропусков, выберем изначально импутацию модами,
# обозначив 'unknown' в качестве пропущенных значений,
# TargetEncoder - для кодирования переменных с 
# помощью среднего значения целевого признака

cat_preprocessor = Pipeline([                 
    ('rcg', RareLabelCategoricalEncoder(tol=tol,
                                        n_categories=2,
                                        replace_with='Rare')),
    ('imp', PandasSimpleImputer(strategy='most_frequent',
                                missing_values='unknown')),
    ('enc', ce.CatBoostEncoder(cols=categorical))                          
])


# для числовых переменных -

# MinMaxScaler - для приведения переменных к одному масштабу
# SimpleImputer - для обработки пропусков, выберем импутацию модами
num_preprocessor = Pipeline([
    ('scaler', MinMaxScaler()),
    ('imp', PandasSimpleImputer(strategy='most_frequent', 
                                fill_value=-999999999)),                       
])


# объединим их с помощью ColumnTransformer
base_preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, categorical),
    ('num', num_preprocessor, numerical)
])

In [25]:
# задаем список бэйзлайн-моделей
baseline_models = [
    BernoulliNB(),
    GaussianNB(),
    KNeighborsClassifier(n_jobs=-1),
    LogisticRegression(random_state=SEED, n_jobs=-1),
    RandomForestClassifier(random_state=SEED, n_jobs=-1), 
    ExtraTreesClassifier(random_state=SEED, n_jobs=-1), 
    AdaBoostClassifier(random_state=SEED),
    XGBClassifier(random_state=SEED, tree_method='gpu_hist'),
    LGBMClassifier(random_state=SEED),
    CatBoostClassifier(learning_rate=0.1, n_estimators=100,
                       random_state=SEED, task_type="GPU"),
]

In [26]:
categorical_idxs = [train.columns.get_loc(c) for c in categorical]

In [27]:
# создадим общий словарик для всех моделей
models = dict()

# для каждой модели
for model_idx, model in enumerate(baseline_models):

    # создаем ветку модели в общем словаре
    model_name = model.__class__.__name__
    models[model_name] = dict()
    
    # делаем точную копию пайплайна препроцессинга
    preprocessor = deepcopy(base_preprocessor)
    
    # если моделью является CatBoost
    if model_name == 'CatBoostClassifier':

        # нужно отключить encoder внутри пайплайна предобработки
        # категориальных переменных, т.к. будет использоваться
        # встроенный в CatBoostClassifier способ кодирования переменных
        preprocessor.set_params(**{'cat__enc': 'passthrough'})

        # передаем в параметры обучения индексы категориальных переменных
        fit_params = {'model__cat_features': categorical_idxs}

    # для всех остальных моделей
    else:

        # параметры обучения оставляем пустыми
        fit_params = None
        
    # создаем полный пайплайн (препроцессинг -> модель)
    pipe = Pipeline([('preprocessor', preprocessor),
                      ('model', model)])

    # проводим перекрестную проверку
    scores = cross_val_score(pipe, X_train, y_train, 
                             cv=cv, scoring='roc_auc',
                             error_score='raise', 
                             fit_params=fit_params,
                             n_jobs=-1)

    # сохраним в ветке модели объект полного пайплайна
    # для того, чтобы использовать его в дальнейшем
    models[model_name]['pipe'] = pipe

    # записываем cv-оценки под ключем actual
    # для того, чтобы сравнивать любые дальнейшие 
    # изменения с оценками бэйзлайна
    models[model_name]['actual'] = scores
    
    # печатаем статистики полученных оценок
    model_message = f'{model_name} - '
    margin = ' ' * len(model_message)
    message = (model_message 
               + f'cv-scores: {np.round(scores, 4)},\n' 
               + margin 
               + f'shapiro_pvalue = {ss.shapiro(scores)[1]:.2f}\n' 
               + margin 
               + f'mean score = {scores.mean():.4f}\n' 
               + margin 
               + f'0.95 confint: {tconfint_mean(scores)}\n')
    print(message)

BernoulliNB - cv-scores: [0.638  0.6239 0.6227 0.6156 0.6204],
              shapiro_pvalue = 0.31
              mean score = 0.6241
              0.95 confint: [0.6137 0.6345]

GaussianNB - cv-scores: [0.6356 0.6467 0.6375 0.6356 0.639 ],
             shapiro_pvalue = 0.07
             mean score = 0.6389
             0.95 confint: [0.6332 0.6446]

KNeighborsClassifier - cv-scores: [0.5404 0.5555 0.5624 0.5699 0.5565],
                       shapiro_pvalue = 0.78
                       mean score = 0.5569
                       0.95 confint: [0.5434 0.5705]

LogisticRegression - cv-scores: [0.7147 0.6984 0.7118 0.7051 0.699 ],
                     shapiro_pvalue = 0.38
                     mean score = 0.7058
                     0.95 confint: [0.6966 0.715 ]

RandomForestClassifier - cv-scores: [0.6833 0.686  0.6758 0.6877 0.6578],
                         shapiro_pvalue = 0.14
                         mean score = 0.6781
                         0.95 confint: [0.6629 0.6933]

ExtraT

По итогам проделанной проверки мною были выбраны две базовые модели для дальнейшей настройки, дающие наибольшее качество на кросс-валидации - XGBClassifier и CatBoostClassifier.

In [ ]:
models = {model: models[model] for model in ['XGBClassifier', 'CatBoostClassifier']}

In [ ]:
# сохраняем словарь для дальнейшей настройки параметров
save_obj(models, '../data/models_dictionary/models_baseline.pkl')